In [1]:
import getpass

OPENAI_MODEL_NAME = "gpt-35-turbo"
OPENAI_DEPLOYMENT_ID = "chat"
OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = "2023-09-01-preview"

# "https://<your-endpoint.openai.azure.com/"
OPENAI_API_BASE = input('OpenAI API Base:')
OPENAI_API_KEY = getpass.getpass('OpenAI API Key:')

OpenAI API Base: https://aoai-r7a5whblfnou2.openai.azure.com/
OpenAI API Key: ········


In [2]:
# Chat model
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name=OPENAI_MODEL_NAME,
    openai_api_base=OPENAI_API_BASE,
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_kwargs={
        "deployment_id": OPENAI_DEPLOYMENT_ID,
        "api_type": OPENAI_API_TYPE,
        "api_version": OPENAI_API_VERSION,
    },
)

## [LangChain の評価機能を試す](https://note.com/npaka/n/nee310c6bd700)

In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

# 質問応答のプロンプトテンプレートの準備
prompt = PromptTemplate(
    template="Question: {question}\nAnswer:", 
    input_variables=["question"]
)

# 質問応答のLLMチェーンの準備
chain = LLMChain(
    llm=llm, 
    prompt=prompt,
)

In [4]:
# 評価データセットの準備
examples = [
    {
        "question": "テニスボールが2ケースあります。1ケースにテニスボールが3個入っています。今何個のテニスボールを持っていますか？",
        "answer": "6"
    },
    {
        "question": '次の文はもっともらしいですか？もっともらしくないですか？ "カレーは飲み物"',
        "answer": "もっともらしくない"
    }
]

In [5]:
# 予測
predictions = chain.apply(examples)
predictions

[{'text': '6個のテニスボールを持っています。'}, {'text': 'もっともらしくないです。'}]

In [6]:
from langchain.evaluation.qa import QAEvalChain

# 言語モデルによる評価
eval_chain = QAEvalChain.from_llm(
    llm=llm,
)
graded_outputs = eval_chain.evaluate(
    examples, 
    predictions, 
    question_key="question", 
    prediction_key="text"
)
print(graded_outputs)

[{'results': 'CORRECT'}, {'results': 'CORRECT'}]


In [7]:
# 結果を整形して出力
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + eg['question'])
    print("Real Answer: " + eg['answer'])
    print("Predicted Answer: " + predictions[i]['text'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: テニスボールが2ケースあります。1ケースにテニスボールが3個入っています。今何個のテニスボールを持っていますか？
Real Answer: 6
Predicted Answer: 6個のテニスボールを持っています。
Predicted Grade: CORRECT

Example 1:
Question: 次の文はもっともらしいですか？もっともらしくないですか？ "カレーは飲み物"
Real Answer: もっともらしくない
Predicted Answer: もっともらしくないです。
Predicted Grade: CORRECT

